In [5]:
tf="training_label.txt"
with open(tf) as f:
    tfl=f.readlines()
#print(tfl)
outlist=[]
for i in tfl:
    outlist.append(i[:-1])
# print(outlist,'/n')


In [3]:
import pandas as pd
import os
import torch

In [4]:
#读取txt文本
df =  pd.read_table(tf,sep='\t',names=['name','label'])
print(df)
#然后选定label列下的数据，进行绘图
# 

           name  label
0       0_0.jpg      0
1       0_1.jpg      0
2      0_10.jpg      0
3     0_100.jpg      0
4     0_101.jpg      0
...         ...    ...
9861  9_995.jpg      9
9862  9_996.jpg      9
9863  9_997.jpg      9
9864  9_998.jpg      9
9865  9_999.jpg      9

[9866 rows x 2 columns]


In [6]:
from sklearn.utils import shuffle
import numpy as np
def labelShuffling(dataFrame, groupByName='label'):
    groupDataFrame = dataFrame.groupby(by=[groupByName])
    labels = groupDataFrame.size()
    print("length of label is ", len(labels))
    maxNum = max(labels)
    lst = pd.DataFrame()
    for i in range(len(labels)):
        print("Processing label  :", i)
        tmpGroupBy = groupDataFrame.get_group(i)
        createdShuffleLabels = np.random.permutation(np.array(range(maxNum))) % labels[i]  # 随机排列组合
        print("Num of the label is : ", labels[i])
        lst=lst.append(tmpGroupBy.iloc[createdShuffleLabels], ignore_index=True)
        # print("Done")
    # lst.to_csv('test1.csv', index=False)
    return lst

all_size = len(df)
print("训练集大小：", all_size)

# train_image_list = df

df1 = labelShuffling(df)
df1 = shuffle(df1)
print("shuffle后数据集大小：", len(df1))

# train_image_path_list = df1['name'].values
# label_list = df1['label'].values
# label_list = torch.to_tensor(label_list, dtype='int64')
# train_label_list = torch.nn.functional.one_hot(label_list, num_classes=11)


训练集大小： 9866
length of label is  11
Processing label  : 0
Num of the label is :  994
Processing label  : 1
Num of the label is :  429
Processing label  : 2
Num of the label is :  1500
Processing label  : 3
Num of the label is :  986
Processing label  : 4
Num of the label is :  848
Processing label  : 5
Num of the label is :  1325
Processing label  : 6
Num of the label is :  440
Processing label  : 7
Num of the label is :  280
Processing label  : 8
Num of the label is :  855
Processing label  : 9
Num of the label is :  1500
Processing label  : 10
Num of the label is :  709
shuffle后数据集大小： 16500


In [ ]:
file_path = 'training/'
f_test=open('training_label1.txt','a')
n=len(df1)
#写入txt中

for _,_,file in os.walk(file_path):
    for f in file:
        tmp=f.split('_')[0]
        f_test.write(f+'\t'+tmp+'\n')

In [11]:
import torch
import pandas as pd
import os
from skimage import io, transform
from PIL import Image
import cv2
import numpy as np
class Mydataset(torch.utils.data.Dataset):
    def __init__(self,label_file,picture_dir,transform=None):
        
        # self.label=pd.read_table(label_file,sep='\t',names=['name','label'])
        self.label=label_file
        self.image_dir=picture_dir
        self.transform=transform
    
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx=idx.tolist()
        img_name=os.path.join(self.image_dir,self.label.iloc[idx,0])
        # image=io.imread(img_name)
        # image=cv2.imread(img_name)
        image=Image.open(img_name).convert("RGB")
        # image = Image.open(img_name).convert('RGB')
        image = np.array(image).astype('float32')
        image = transform.resize(image,(100,100))
        image = torch.tensor(image, dtype=torch.float32)
        # print(type(image))
        label=self.label.iloc[idx,1]
        sample = {'image':image,'label':label}
        if self.transform:
            sample=self.transform(sample)
        return sample

In [13]:
train_dataset_optimized = Mydataset(df1,'training')
trainloader_optimized = torch.utils.data.DataLoader(train_dataset_optimized, batch_size=4)

In [14]:
for i, data in enumerate(trainloader_optimized, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['image'],data['label']
        print(type(inputs))
        print(i)
        print(labels)
        break

<class 'torch.Tensor'>
0
tensor([6, 3, 5, 0])


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet,self).__init__()
        self.conv0=nn.Conv2d(3,20,5,padding=0)
        self.pool0=nn.MaxPool2d(kernel_size=2,stride=2)
        self._batch_norm_0 = nn.BatchNorm2d(num_features=20)

        self.conv1 = nn.Conv2d(20,50,kernel_size=5,padding=0)
        self.pool1=nn.MaxPool2d(kernel_size=2,stride=2)
        self._batch_norm_1= nn.BatchNorm2d(num_features=50)

        self.conv2=nn.Conv2d(50,50,kernel_size=5,padding=0)
        self.pool2=nn.MaxPool2d(kernel_size=2,stride=2)
        self.fc1=nn.Linear(in_features=4050,out_features=218)
        self.fc2=nn.Linear(in_features=218,out_features=100)
        self.fc3=nn.Linear(in_features=100,out_features=11)
    def forward(self,input):
        input=input.reshape(-1,3,100,100)
        x = self.conv0(input)  #数据输入卷积层
        x = F.relu(x)  # 激活层
        x = self.pool0(x)  # 池化层
        x = self._batch_norm_0(x)  # 归一层

        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self._batch_norm_1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = x.reshape(x.shape[0], -1)
        # print(x.shape)

        x = self.fc1(x)  # 线性层
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        y = F.softmax(x)  # 分类器
        return y


net = MyNet()
print(net)

MyNet(
  (conv0): Conv2d(3, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (_batch_norm_0): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (_batch_norm_1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(50, 50, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4050, out_features=218, bias=True)
  (fc2): Linear(in_features=218, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=11, bias=True)
)


In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [32]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader_optimized, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data['image'],data['label']

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')


C:\Users\admin\AppData\Local\Temp/ipykernel_52040/1702851010.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(x)  # 分类器


[1,  1000] loss: 2.287
[1,  2000] loss: 2.288
[1,  3000] loss: 2.284
[1,  4000] loss: 2.276
[2,  1000] loss: 2.270
[2,  2000] loss: 2.264
[2,  3000] loss: 2.260
[2,  4000] loss: 2.248
[3,  1000] loss: 2.252
[3,  2000] loss: 2.237
[3,  3000] loss: 2.241
[3,  4000] loss: 2.230
[4,  1000] loss: 2.221
[4,  2000] loss: 2.214
[4,  3000] loss: 2.221
[4,  4000] loss: 2.207
[5,  1000] loss: 2.213
[5,  2000] loss: 2.193
[5,  3000] loss: 2.196
[5,  4000] loss: 2.188
[6,  1000] loss: 2.190
[6,  2000] loss: 2.171
[6,  3000] loss: 2.183
[6,  4000] loss: 2.173
[7,  1000] loss: 2.172
[7,  2000] loss: 2.153
[7,  3000] loss: 2.155
[7,  4000] loss: 2.150
[8,  1000] loss: 2.140
[8,  2000] loss: 2.130
[8,  3000] loss: 2.135
[8,  4000] loss: 2.128
[9,  1000] loss: 2.126
[9,  2000] loss: 2.114
[9,  3000] loss: 2.107
[9,  4000] loss: 2.108
[10,  1000] loss: 2.104
[10,  2000] loss: 2.092
[10,  3000] loss: 2.087
[10,  4000] loss: 2.092
[11,  1000] loss: 2.085
[11,  2000] loss: 2.071
[11,  3000] loss: 2.069
[11,

In [19]:
PATH = './tmp_net2.pth'
torch.save(net.state_dict(), PATH)

In [31]:
img=Image.open("1.png").convert("RGB")
image = np.array(img).astype('float32')
image = transform.resize(image,(100,100))
print(image.size)
image = torch.tensor(image, dtype=torch.float32)
outputs=net(image)
_,predicted = torch.max(outputs,1)
predicted

30000


C:\Users\admin\AppData\Local\Temp/ipykernel_52040/1702851010.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(x)  # 分类器


tensor([10])

In [22]:
df_test=pd.read_table("testing_label.txt",sep='\t',names=['name','label'])
test_dataset = Mydataset(label_file=df_test,picture_dir='validation/')
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=4)

In [23]:
import torchvision
import matplotlib.pyplot as plt
dataiter = iter(testloader)
tmp=dataiter.next()
images, labels =tmp['image'],tmp['label']
classes=['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles/Pasta', 'Rice', 'Seafood', 'Soup','Vegetable/Fruit']


In [24]:
dataiter = iter(testloader)
tmp=dataiter.next()
sumnum_2=0
sumnum_1=0
for i,data in enumerate(testloader):
    images,labels = data['image'],data['label']
    outputs=net(images)
    _,predicted = torch.max(outputs,1)
    mask=predicted==labels
    sumnum=sum(mask)
    sumnum_1+=sumnum_1
    sumnum_2=sumnum_2+4
    if i==1000:break
print(sumnum_1/sumnum_2)
print(sumnum_1)
print(sumnum_2)

C:\Users\admin\AppData\Local\Temp/ipykernel_52040/1702851010.py:44: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(x)  # 分类器


0.0
0
3432
